Origem dos datasets

ISIC:

* https://gallery.isic-archive.com/#!/topWithHeader/onlyHeaderTop/gallery?filter=%5B%22mel_class%7Cmelanoma%20in%20situ%22%5D
* https://gallery.isic-archive.com/#!/topWithHeader/onlyHeaderTop/gallery?filter=%5B%22diagnosis%7Cangioma%22%2C%22diagnosis%7CAIMP%22%5D&name=

Kaggle:

* https://www.kaggle.com/datasets/cdeotte/jpeg-melanoma-512x512?resource=download

Importando as dependências

In [2]:
import torch
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import io
import shutil
import sklearn
import torchvision
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models.detection import ssdlite320_mobilenet_v3_large

Redução do volume de dados no dataser Kaggle

In [5]:
def reduzir_imagens(diretorio, quantidade_desejada):
    # Obter lista de arquivos
    arquivos = [f for f in os.listdir(diretorio) if os.path.isfile(os.path.join(diretorio, f))]
    
    # Verificar a quantidade de imagens
    if len(arquivos) > quantidade_desejada:
        # Selecionar aleatoriamente
        arquivos_para_manter = random.sample(arquivos, quantidade_desejada)
        arquivos_para_remover = set(arquivos) - set(arquivos_para_manter)
        
        # Remover as imagens
        for arquivo in arquivos_para_remover:
            caminho_arquivo = os.path.join(diretorio, arquivo)
            os.remove(caminho_arquivo)
    else:
        print(f"O diretório {diretorio} já possui {len(arquivos)} ou menos arquivos.")

diretorio_train = "/home/abel/Downloads/TCC-PUC-Melanoma-Detection/archive/train"
diretorio_test = "/home/abel/Downloads/TCC-PUC-Melanoma-Detection/archive/test"

# Reduzir imagens nos diretórios
reduzir_imagens(diretorio_train, 1326)
reduzir_imagens(diretorio_test, 440)

Unificando os datasets de malanoma (Kaggle e ISIC melanoma)

In [6]:
# Caminho dos datasets
isic_dir = '/home/abel/Downloads/TCC-PUC-Melanoma-Detection/ISIC-images'  # Caminho correto
archive_dir = '/home/abel/Downloads/TCC-PUC-Melanoma-Detection/archive'   # Caminho correto
output_dir = '/home/abel/Downloads/TCC-PUC-Melanoma-Detection/dataset'  # Diretório de saída

# Verificar se os diretórios existem
if not os.path.exists(isic_dir):
    raise FileNotFoundError(f"O diretório {isic_dir} não existe. Verifique o caminho.")
if not os.path.exists(archive_dir):
    raise FileNotFoundError(f"O diretório {archive_dir} não existe. Verifique o caminho.")

os.makedirs(output_dir, exist_ok=True)

image_output_dir = os.path.join(output_dir, 'melanoma')
os.makedirs(image_output_dir, exist_ok=True)

# Unificar imagens do dataset ISIC
isic_metadata_path = os.path.join(isic_dir, 'metadata.csv')

if not os.path.exists(isic_metadata_path):
    raise FileNotFoundError(f"O arquivo de metadados {isic_metadata_path} não foi encontrado. Verifique o caminho.")

isic_metadata = pd.read_csv(isic_metadata_path)

# Verificar imagens ausentes do dataset ISIC
imagens_ausentes_isic = isic_metadata[~isic_metadata['isic_id'].apply(lambda x: os.path.exists(os.path.join(isic_dir, f'{x}.jpg')))]
if len(imagens_ausentes_isic) > 0:
    print(f"Imagens ausentes no dataset ISIC: {len(imagens_ausentes_isic)}")

# Filtrar metadados para verificar apenas as imagens existentes no diretório
isic_metadata = isic_metadata[isic_metadata['isic_id'].apply(lambda x: os.path.exists(os.path.join(isic_dir, f'{x}.jpg')))]

# Copiar as imagens para o local do dataset
for image_name in isic_metadata['isic_id']:
    src_path = os.path.join(isic_dir, f'{image_name}.jpg')
    dest_path = os.path.join(image_output_dir, f'{image_name}.jpg')
    shutil.copy(src_path, dest_path)

isic_metadata['image_path'] = isic_metadata['isic_id'].apply(lambda x: os.path.join(image_output_dir, f'{x}.jpg'))

# Unificar imagens do dataset Kaggle
archive_train_metadata_path = os.path.join(archive_dir, 'train.csv')
archive_test_metadata_path = os.path.join(archive_dir, 'test.csv')

archive_train_metadata = pd.read_csv(archive_train_metadata_path)
archive_test_metadata = pd.read_csv(archive_test_metadata_path)

# Identificar colunas comuns
common_columns = list(set(archive_train_metadata.columns) & set(archive_test_metadata.columns))

# Manter apenas colunas comuns
archive_train_metadata = archive_train_metadata[common_columns]
archive_test_metadata = archive_test_metadata[common_columns]

# Verificar imagens ausentes no dataset Kaggle (train e test)
imagens_ausentes_train = archive_train_metadata[~archive_train_metadata['image_name'].apply(lambda x: os.path.exists(os.path.join(archive_dir, 'train', f'{x}.jpg')))]
imagens_ausentes_test = archive_test_metadata[~archive_test_metadata['image_name'].apply(lambda x: os.path.exists(os.path.join(archive_dir, 'test', f'{x}.jpg')))]
if len(imagens_ausentes_train) > 0:
    print(f"Imagens ausentes no dataset Kaggle (train): {len(imagens_ausentes_train)}")
if len(imagens_ausentes_test) > 0:
    print(f"Imagens ausentes no dataset Kaggle (test): {len(imagens_ausentes_test)}")

# Filtrar imagens existentes
archive_train_metadata = archive_train_metadata[archive_train_metadata['image_name'].apply(lambda x: os.path.exists(os.path.join(archive_dir, 'train', f'{x}.jpg')))]
archive_test_metadata = archive_test_metadata[archive_test_metadata['image_name'].apply(lambda x: os.path.exists(os.path.join(archive_dir, 'test', f'{x}.jpg')))]

# Copiar imagens do treino
for image_name in archive_train_metadata['image_name']:
    src_path = os.path.join(archive_dir, 'train', f'{image_name}.jpg')
    dest_path = os.path.join(image_output_dir, f'{image_name}.jpg')
    shutil.copy(src_path, dest_path)

# Copiar imagens do teste
for image_name in archive_test_metadata['image_name']:
    src_path = os.path.join(archive_dir, 'test', f'{image_name}.jpg')
    dest_path = os.path.join(image_output_dir, f'{image_name}.jpg')
    shutil.copy(src_path, dest_path)

archive_train_metadata['image_path'] = archive_train_metadata['image_name'].apply(lambda x: os.path.join(image_output_dir, f'{x}.jpg'))
archive_test_metadata['image_path'] = archive_test_metadata['image_name'].apply(lambda x: os.path.join(image_output_dir, f'{x}.jpg'))

# Combinar os metadados
for col in set(isic_metadata.columns) - set(common_columns):
    archive_test_metadata[col] = pd.NA

combined_metadata = pd.concat([isic_metadata, archive_train_metadata, archive_test_metadata], ignore_index=True)

# Tratar imagens duplicadas
duplicatas = combined_metadata.duplicated(subset='image_path', keep=False)
imagens_duplicadas = combined_metadata[duplicatas]
if len(imagens_duplicadas) > 0:
    print(f"Imagens duplicadas encontradas: {len(imagens_duplicadas)}")
    # Mantemos apenas a primeira ocorrência de cada imagem duplicada
    combined_metadata = combined_metadata.drop_duplicates(subset='image_path', keep='first')

# Salvar os metadados combinados
combined_metadata_path = os.path.join(output_dir, 'metadata.csv')
combined_metadata.to_csv(combined_metadata_path, index=False)

total_imagens = len(combined_metadata)
print(f"Total de imagens no novo dataset: {total_imagens}")

print("Unificação completa. Os arquivos estão salvos em:", output_dir)

Imagens ausentes no dataset Kaggle (train): 31800
Imagens ausentes no dataset Kaggle (test): 10542
Imagens duplicadas encontradas: 450
Total de imagens no novo dataset: 2100
Unificação completa. Os arquivos estão salvos em: /home/abel/Downloads/TCC-PUC-Melanoma-Detection/dataset


In [78]:
from PIL import Image
import os

#Redimensiona as imagens para ficar com tamanhos iguais

def resize_images_in_directory(directory, size=(512, 512)):
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
            image_path = os.path.join(directory, filename)
            with Image.open(image_path) as img:
                img = img.resize(size, Image.LANCZOS)
                img.save(image_path)
            
resize_images_in_directory('/home/abel/Downloads/TCC-PUC-Melanoma-Detection/dataset/melanoma')
resize_images_in_directory('/home/abel/Downloads/TCC-PUC-Melanoma-Detection/dataset/not_melanoma')

Preparando o dataset para treinamento, validação e teste

In [85]:
from sklearn.model_selection import train_test_split

# Diretório do dataset
base_dir = '/home/abel/Downloads/TCC-PUC-Melanoma-Detection/dataset'

melanoma_dir = os.path.join(base_dir, 'melanoma')
not_melanoma_dir = os.path.join(base_dir, 'not_melanoma')

# Criar diretórios de train, val, test
os.makedirs(os.path.join(base_dir, 'train', 'melanoma'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'train', 'not_melanoma'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'val', 'melanoma'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'val', 'not_melanoma'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'test', 'melanoma'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'test', 'not_melanoma'), exist_ok=True)

# Mover os arquivos
def move_files(file_list, source_dir, target_dir):
    for file_name in file_list:
        shutil.move(os.path.join(source_dir, file_name), os.path.join(target_dir, file_name))

melanoma_files = os.listdir(melanoma_dir)
not_melanoma_files = os.listdir(not_melanoma_dir)

# Dividir os arquivos em train, val, test (80%, 10%, 10%)
melanoma_train, melanoma_temp = train_test_split(melanoma_files, test_size=0.2, random_state=42)
melanoma_val, melanoma_test = train_test_split(melanoma_temp, test_size=0.5, random_state=42)

not_melanoma_train, not_melanoma_temp = train_test_split(not_melanoma_files, test_size=0.2, random_state=42)
not_melanoma_val, not_melanoma_test = train_test_split(not_melanoma_temp, test_size=0.5, random_state=42)

move_files(melanoma_train, melanoma_dir, os.path.join(base_dir, 'train', 'melanoma'))
move_files(melanoma_val, melanoma_dir, os.path.join(base_dir, 'val', 'melanoma'))
move_files(melanoma_test, melanoma_dir, os.path.join(base_dir, 'test', 'melanoma'))

move_files(not_melanoma_train, not_melanoma_dir, os.path.join(base_dir, 'train', 'not_melanoma'))
move_files(not_melanoma_val, not_melanoma_dir, os.path.join(base_dir, 'val', 'not_melanoma'))
move_files(not_melanoma_test, not_melanoma_dir, os.path.join(base_dir, 'test', 'not_melanoma'))

print("Imagens organizadas em pastas para treino, validação e teste!")

Imagens organizadas em pastas para treino, validação e teste!
